In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm

In [2]:
path_to_df="v2_raw_28apr_28may_2021_eth_raw.hdf"
path_to_df_eth="v2_lagging_B28apr_28may_2021_eth.hdf"
path_to_df_ada="v2_lagging_B28apr_28may_2021_ada.hdf"

Apath_to_df_eth="v2_lagging_A22apr_22may_2021_eth.hdf"
Apath_to_df_ada="v2_lagging_A22apr_22may_2021_ada.hdf"

Bpath_to_df_eth="v2_lagging_B22apr_22may_2021_eth.hdf"
Bpath_to_df_ada="v2_lagging_B22apr_22may_2021_ada.hdf"

df = pd.read_hdf(path_to_df_ada)
df

,trade_in_3h_usdtfutures_ADAUSDT_close_price,trade_in_2h_usdtfutures_ADAUSDT_close_price,trade_in_1h_usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_volume,usdtfutures_ADAUSDT_number_of_trades,SMA(usdtfutures_ADAUSDT_close_price_5),SMA(usdtfutures_ADAUSDT_volume_5),SMA(usdtfutures_ADAUSDT_number_of_trades_5),SMA(usdtfutures_ADAUSDT_close_price_30),...,SMA(usdtfutures_ADAUSDT_volume_30),shift1_SMA(usdtfutures_ADAUSDT_volume_30),SMA(usdtfutures_ADAUSDT_number_of_trades_30),shift1_SMA(usdtfutures_ADAUSDT_number_of_trades_30),SMA(usdtfutures_ADAUSDT_close_price_60),shift1_SMA(usdtfutures_ADAUSDT_close_price_60),SMA(usdtfutures_ADAUSDT_volume_60),shift1_SMA(usdtfutures_ADAUSDT_volume_60),SMA(usdtfutures_ADAUSDT_number_of_trades_60),shift1_SMA(usdtfutures_ADAUSDT_number_of_trades_60)
2021-04-28 04:30:59.999000+00:00,-0.024310,-0.041629,-0.013785,-0.002802,0.148918,-0.038462,0.000062,0.167022,0.091397,0.000193,...,0.113760,0.335982,0.010926,0.190058,-2.596959e-05,-0.000256,0.224871,0.383095,0.100492,0.182311
2021-04-28 04:31:59.999000+00:00,-0.024254,-0.043951,-0.014630,-0.000518,-0.042713,-0.040000,0.000008,0.200485,0.153134,0.000044,...,0.136194,0.330241,0.029672,0.183343,-4.004986e-05,-0.000172,0.233218,0.377304,0.106508,0.178901
2021-04-28 04:32:59.999000+00:00,-0.023677,-0.049662,-0.015043,-0.000069,-0.716952,-0.376812,-0.000309,0.009052,0.003024,0.000161,...,0.105733,0.356198,0.023102,0.196836,-6.728834e-05,-0.000176,0.230965,0.351996,0.109969,0.150497
2021-04-28 04:33:59.999000+00:00,-0.025454,-0.044463,-0.018917,0.000213,0.531599,-0.072674,-0.000226,0.067308,-0.009550,0.000189,...,0.144847,0.333677,0.036907,0.188812,-7.283867e-05,-0.000226,0.239262,0.350473,0.112859,0.152630
2021-04-28 04:34:59.999000+00:00,-0.023670,-0.047592,-0.017454,0.000053,0.498859,0.206897,-0.000624,0.083942,-0.064210,0.000200,...,0.151970,0.346127,0.037479,0.178125,-9.950721e-05,-0.000255,0.249049,0.361705,0.107802,0.160682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-28 20:38:59.999000+00:00,-0.015252,-0.032970,-0.047183,0.001430,-0.436621,-0.294782,-0.000943,-0.159298,-0.107821,-0.000347,...,0.163005,0.106905,0.079824,0.027768,7.016943e-05,-0.000048,0.134955,0.154300,0.053796,0.063201
2021-05-28 20:39:59.999000+00:00,-0.012764,-0.030542,-0.044152,-0.003375,1.312584,0.788711,-0.000830,0.098967,0.030063,-0.000483,...,0.219553,0.108669,0.111563,0.031289,2.152401e-05,-0.000005,0.164111,0.155779,0.071426,0.063744
2021-05-28 20:40:59.999000+00:00,-0.011341,-0.033372,-0.044909,-0.000912,-0.457343,-0.317271,-0.000649,0.076769,0.043417,-0.000537,...,0.207247,0.090964,0.102107,0.026924,-2.735412e-05,0.000029,0.149105,0.145441,0.064516,0.056563
2021-05-28 20:41:59.999000+00:00,-0.015236,-0.037049,-0.051830,0.001043,-0.095362,-0.063710,-0.000233,0.119102,0.051954,-0.000474,...,0.173626,0.133323,0.088643,0.045496,8.709940e-07,0.000022,0.153475,0.161339,0.067069,0.065779


In [3]:
def check_x_for_leak(X):
    for column in X.columns:
        assert "trade_in_" not in column
    
    

In [4]:
def create_model(path_to_hdf, index, train_size, test_size, coin):
    df_orig = pd.read_hdf(path_to_hdf)
    df_orig = df_orig.fillna(0)
    del df_orig["trade_in_3h_usdtfutures_{}USDT_close_price".format(coin)]
    del df_orig["trade_in_2h_usdtfutures_{}USDT_close_price".format(coin)]
    start = index - (train_size+test_size)
    end = index
    df_orig = df_orig[start:end]

    df = df_orig.copy()
    Y = df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)]

    Y_MEAN = np.mean(np.abs(Y))
    Y_MEAN = Y_MEAN * (1/4)

    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] > Y_MEAN] = 20
    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] < -Y_MEAN] = 10
    Y[df["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)] < 9] = 1

    X = df_orig.loc[:, "usdtfutures_{}USDT_close_price".format(coin) :]
    check_x_for_leak(X)

    test_size = test_size / (test_size+train_size)
    test_size = test_size or 0.01
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=test_size, random_state=6, shuffle=False
    )
    hours_in_test = len(Y_test)

    model = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        booster="gbtree",
        gamma=0.1,
        sampling_method=["uniform", "gradient_based"][0],
        num_parallel_tree=1,
        reg_alpha=1,
        reg_lambda=0,
        eta=0.3,
    )
    model.fit(X_train, Y_train)
    return model
    

In [5]:
def predict_row(path_to_hdf, index, model, coin):
    df_orig = pd.read_hdf(path_to_hdf)
    df_orig = df_orig.fillna(0)
    del df_orig["trade_in_3h_usdtfutures_{}USDT_close_price".format(coin)]
    del df_orig["trade_in_2h_usdtfutures_{}USDT_close_price".format(coin)]
    df_orig = df_orig.iloc[index:index+1]
    
    X = df_orig.loc[:,"usdtfutures_{}USDT_close_price".format(coin) :]
    check_x_for_leak(X)
    y_pred = model.predict(X)[0]
    Y_TEST_REAL = df_orig["trade_in_1h_usdtfutures_{}USDT_close_price".format(coin)][0]
    
    profit = -1
    
    side = -1
    if y_pred == 20:
        side = 1
    if y_pred == 1:
        return 0

    if side * Y_TEST_REAL > 0:
        profit = 1
    
    return abs(Y_TEST_REAL) * profit
    
    

    
    

In [6]:
predicts_adaA = []
predicts_ethA = []
predicts_adaB = []
predicts_ethB = []
model_c = 11
model=None
for i in tqdm.tqdm(range(60*24*6, 60*24*10, 60)):
#for i in tqdm.tqdm(range(41000, 44289, 60)):
    model_c -= 1
    if model_c ==0:
        model_c = 10
    
    if model_c ==10:
        pass
    model_adaA = create_model(Apath_to_df_ada, i, 60*64*1, 60, "ADA")
    predict_adaA = predict_row(Apath_to_df_ada, i, model_adaA, "ADA")
    predicts_adaA.append(predict_adaA)
    #predicts_ethA.append(predict_ethA)
    #predicts_adaB.append(predict_adaB)
    #predicts_ethB.append(predict_ethB)

  0%|          | 0/96 [00:00<?, ?it/s]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:19:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
  1%|          | 1/96 [00:02<04:06,  2.59s/it]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:19:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
  2%|▏         | 2/96 [00:05<04:14,  2.71s/it]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
 98%|█████████▊| 94/96 [06:34<00:06,  3.29s/it]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:25:43] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
 99%|█████████▉| 95/96 [06:37<00:03,  3.12s/it]/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:25:45] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/vaclavmatejka/anaconda3/envs/pybinbacktester/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
100%|██████████| 96/96 [06:40<00:00,  4.17s/it]


In [7]:
initial_bank = 100
rake = 0.00018

for i in range(len(predicts_adaA)):
    if i ==0:
        continue
    
    predicts_lists = [predicts_adaA]
    results = [predicts_adaA[i-1]]
    max_i = np.argmax(results)
    p = predicts_lists[max_i][i]

    change = initial_bank * p
    initial_bank = initial_bank + change
    
    rake_change = initial_bank * rake * -2
    initial_bank = initial_bank + rake_change
    
    
    
    
initial_bank

85.61740746881185